In [269]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib as mpl
from datetime import datetime
import math 
plt.style.use('https://raw.githubusercontent.com/RobGeada/stylelibs/main/material_rh.mplstyle')

In [278]:
df = pd.read_pickle("level_2_results.pkl")
df.columns = ["metric_"+x if "dataset" not in x and "model" not in x and x!="explainer" else x for x in list(df)] 

In [279]:
df

,dataset,dataset_mu,dataset_rho,dataset_dim,dataset_noise,dataset_weight,dataset_num_train_samples,dataset_num_val_samples,model,explainer,metric_runtime,model_perf_train,model_perf_test,metric_roar_faithfulness,metric_roar_monotonicity,metric_faithfulness,metric_monotonicity,metric_shapley,metric_shapley_corr,metric_infidelity
0,gaussianLinear,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,lr,shap_trustyai,19.922076,0.00,0.03,0.595424,0.8250,0.963328,0.4125,0.002042,0.977280,0.009079
1,gaussianLinear,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,lr,kernelshap,19.865959,0.00,0.03,0.595424,0.8250,0.967515,0.4275,0.002042,0.977280,0.008983
2,gaussianLinear,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,lr,lime_trustyai,64.885579,0.00,0.03,0.055340,0.4550,0.063928,0.3700,0.260806,0.979057,0.127435
3,gaussianLinear,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,lr,lime,10.257473,0.00,0.03,0.435326,0.6925,0.772451,0.4000,0.066224,0.876122,0.010966
4,gaussianLinear,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,lr,random,0.000206,0.00,0.03,0.005515,0.5350,0.048020,0.4150,1.276906,-0.013387,0.082896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,gaussianPiecewiseConstant,"[0.0, 0.0, 0.0, 0.0, 0.0]",1.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,mlp,shap_trustyai,18.416223,0.08,0.08,-0.099503,0.4750,0.084292,0.3275,0.016089,-0.082889,0.017914
221,gaussianPiecewiseConstant,"[0.0, 0.0, 0.0, 0.0, 0.0]",1.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,mlp,kernelshap,23.202546,0.08,0.08,0.031966,0.5300,0.049848,0.3350,0.016089,-0.082889,0.017606
222,gaussianPiecewiseConstant,"[0.0, 0.0, 0.0, 0.0, 0.0]",1.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,mlp,lime_trustyai,163.729276,0.08,0.08,0.045586,0.3850,0.153306,0.3200,0.343626,-0.020211,0.011729
223,gaussianPiecewiseConstant,"[0.0, 0.0, 0.0, 0.0, 0.0]",1.0,5,0.01,"[4, 3, 2, 1, 0]",1000,100,mlp,lime,10.802131,0.08,0.08,-0.040776,0.4425,0.072870,0.2900,0.042663,-0.012388,0.016948


In [119]:
metrics = [x for x in list(df) if "metric" in x]
metric_down = ["metric_infidelity", "metric_runtime", "metric_shapley"]
metric_labels = {metric:metric +" ↓" if metric in metric_down else metric +" ↑" for metric in metrics}
metric_labels

{'metric_runtime': 'metric_runtime ↓',
 'metric_roar_faithfulness': 'metric_roar_faithfulness ↑',
 'metric_roar_monotonicity': 'metric_roar_monotonicity ↑',
 'metric_faithfulness': 'metric_faithfulness ↑',
 'metric_monotonicity': 'metric_monotonicity ↑',
 'metric_shapley': 'metric_shapley ↓',
 'metric_shapley_corr': 'metric_shapley_corr ↑',
 'metric_infidelity': 'metric_infidelity ↓'}

In [120]:
# automatically choose subplot dimensions
def auto_plot_count(n):
    pairs = []
    for i in range(1,n+1):
        j = math.ceil(n/i)
        remainder = (i*j) - n
        dist = np.abs(i-j)
        pairs.append((i,j,2*remainder+dist))
    return sorted(pairs, key=lambda x: (x[2]))[0][0:2]

subplots = auto_plot_count(len(metrics))
subplots

(2, 4)

In [280]:
level=2
cmap = mpl.colormaps['viridis']
explainers = df['explainer'].unique()
n_exp = len(explainers)
   
fig = plt.figure(figsize=(16,9))
labels = []
for metric_idx, metric in enumerate(metrics):
    for exp_idx, explainer in enumerate(explainers):
        exp_df = df[df['explainer']==explainer]
        color = cmap(exp_idx/n_exp)
        n_datasets = len(exp_df)
        plt.subplot(subplots[0], subplots[1], metric_idx+1)
        
        xs = []
        ys = []
        for idx,(_, row) in enumerate(exp_df.iterrows()):
            xs.append(exp_idx)
            ys.append(row[metric])
        plt.scatter(xs+((np.random.rand(len(xs))-.5)/5)+.5, ys, c=[color]*len(xs), s=1)
        violin_parts = plt.violinplot(ys, positions=[exp_idx+.5], showmeans=True)
        if metric_idx == 0:
            labels.append([mpatches.Patch(color=color), explainer])
        
        for partname in ('cbars','cmins','cmaxes','cmeans'):
            violin_parts[partname].set_color(color)
        for v_idx, pc in enumerate(violin_parts['bodies']):
            pc.set_facecolor(color)
            pc.set_edgecolor(color)
        colors.append(cmap(exp_idx/n_exp))
    
    ax = plt.gca()
    locs = np.arange(n_exp)
    
    ax.xaxis.set_ticks(locs, minor=True)
    ax.yaxis.set_ticks(np.array(plt.yticks()[0]), minor=True)

    ax.xaxis.set(ticks=locs+.5, ticklabels=explainers)
    ax.yaxis.set(ticks=np.array(plt.yticks()[0]), ticklabels=plt.yticks()[1])
    ax.grid(True, which='minor',axis='x')
    ax.grid(False, which='major',axis='x')
    plt.xticks(rotation=45)
    plt.title(metric_labels[metric].replace("metric_","").replace("_"," ").title())

plt.suptitle("TrustyAI XAIBench L{} @ {}".format(level, datetime.strftime(datetime.now(), "%H:%m %Y-%m-%d")), color='k', fontsize=16)
plt.tight_layout()
fig.legend(*zip(*labels), loc='lower center', bbox_to_anchor=(.5, -.05),ncols=5)
plt.savefig("xai_bench_l{}.png".format(level),bbox_inches='tight')
plt.show()